In [74]:
from glob import glob
import os
import pandas as pd
import plotly.express as px

In [75]:
def get_slice_id(filename):
    """Read slice id from filename."""
    try:
        slice_id = int(os.path.basename(filename).split('.')[0])
    except ValueError:
        slice_id = None

    return slice_id

In [76]:
df = {}
for analysis_fn in glob("../data/z-stacks/**/**/*PyIQ_analyze_out.csv"):
    stack_id = analysis_fn.split('/')[3]

    analysis = pd.read_csv(analysis_fn)

    # It is important that all slices are sorted to see if a statistic goes with slice depth.
    analysis["slice_id"] = analysis.apply(lambda x: get_slice_id(x['Filename']), axis=1)
    analysis = analysis.sort_values(by="slice_id")
    analysis.set_index("slice_id")

    df[stack_id] = analysis["KurtosisNorm"].to_numpy()

In [77]:
for i in range(62):
    try:
        fig = px.line(df[str(i)])
        fig.show()
    except KeyError:
        continue
